다음 데이터 세트를 이용하여 다이아몬드 가격예측(회귀) 

url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/diamonds.csv" diamonds = pd.read_csv(url) 

1. ML (RF, DT, LR) 수행

2. Dense layer만 이용 FNCC 구현

3. 순환 데이터 변환후 CNN 구현



각 단계별로 수행완료후

검사받은 이후에 다음 단계 진행

In [1]:
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

diamonds = pd.read_csv( "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/diamonds.csv")
diamonds

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [3]:
df = diamonds

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

df['cut'] = label_encoder.fit_transform(df['cut']) # label encoding

In [5]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

df['color'] = label_encoder.fit_transform(df['color']) # label encoding

In [6]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

df['clarity'] = label_encoder.fit_transform(df['clarity']) # label encoding

In [7]:
df

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,2,1,3,61.5,55.0,326,3.95,3.98,2.43
1,0.21,3,1,2,59.8,61.0,326,3.89,3.84,2.31
2,0.23,1,1,4,56.9,65.0,327,4.05,4.07,2.31
3,0.29,3,5,5,62.4,58.0,334,4.20,4.23,2.63
4,0.31,1,6,3,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,2,0,2,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,1,0,2,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,4,0,2,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,3,4,3,61.0,58.0,2757,6.15,6.12,3.74


In [8]:
df.columns

Index(['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'x', 'y',
       'z'],
      dtype='object')

In [9]:
#레이블 개수 확인 (숫자)
df['cut'].value_counts()

2    21551
3    13791
4    12082
1     4906
0     1610
Name: cut, dtype: int64

In [10]:
#결측치 확인
df.isnull().sum()

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
price      0
x          0
y          0
z          0
dtype: int64

In [11]:
#결측치 제거
df = df.dropna()
df.isnull().sum()

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
price      0
x          0
y          0
z          0
dtype: int64

In [12]:
#outcome만 제거하고 x를 만든다
X=df.drop('price',axis=1)
X.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,0.23,2,1,3,61.5,55.0,3.95,3.98,2.43
1,0.21,3,1,2,59.8,61.0,3.89,3.84,2.31
2,0.23,1,1,4,56.9,65.0,4.05,4.07,2.31
3,0.29,3,5,5,62.4,58.0,4.20,4.23,2.63
4,0.31,1,6,3,63.3,58.0,4.34,4.35,2.75


In [13]:
#레이블만 남겨둔다.
y=df['price']
y.value_counts()

605      132
802      127
625      126
828      125
776      124
        ... 
8816       1
14704      1
14699      1
14698      1
9793       1
Name: price, Length: 11602, dtype: int64

In [15]:
Y = pd.get_dummies(y).values

In [16]:
X=X.values

In [17]:
X

array([[0.23, 2.  , 1.  , ..., 3.95, 3.98, 2.43],
       [0.21, 3.  , 1.  , ..., 3.89, 3.84, 2.31],
       [0.23, 1.  , 1.  , ..., 4.05, 4.07, 2.31],
       ...,
       [0.7 , 4.  , 0.  , ..., 5.66, 5.68, 3.56],
       [0.86, 3.  , 4.  , ..., 6.15, 6.12, 3.74],
       [0.75, 2.  , 0.  , ..., 5.83, 5.87, 3.64]])

In [18]:
y

0         326
1         326
2         327
3         334
4         335
         ... 
53935    2757
53936    2757
53937    2757
53938    2757
53939    2757
Name: price, Length: 53940, dtype: int64

In [19]:
from sklearn.model_selection import train_test_split

X_train,X_test, y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=0) 

In [20]:

X_train.shape, y_train.shape, X_test.shape, y_test.shape, 

((43152, 9), (43152, 11602), (10788, 9), (10788, 11602))

In [25]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD,Adam

In [29]:
#Defining the model 

from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam


input_layer = Input(shape=(9,))

hidden1 = Dense(7, activation='tanh')(input_layer)
hidden2 = Dense(64, activation='tanh')(hidden1)
hidden3 = Dense(32, activation='tanh')(hidden2)

output_layer = Dense(1)(hidden3)

model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer='Adam', loss='mse', metrics=['mse'])

model.summary()

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 9)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 7)                   │              70 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 64)                  │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,695 (10.53 KB)

 Trainable params: 2,695 (10.53 KB)

 Non-trainable params: 0 (0.00 B)

In [30]:
N_EPOCHS = 10

history=model.fit(X_train, y_train, epochs=N_EPOCHS, batch_size=32,validation_data= (X_test,y_test))
y_pred = model.predict(X_test)

Epoch 1/10
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - loss: 0.0036 - mse: 0.0036 - val_loss: 8.6185e-05 - val_mse: 8.6185e-05
Epoch 2/10
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 8.6184e-05 - mse: 8.6184e-05 - val_loss: 8.6185e-05 - val_mse: 8.6185e-05
Epoch 3/10
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 8.6184e-05 - mse: 8.6184e-05 - val_loss: 8.6185e-05 - val_mse: 8.6185e-05
Epoch 4/10
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - loss: 2.2594e-04 - mse: 2.2594e-04 - val_loss: 8.6185e-05 - val_mse: 8.6185e-05
Epoch 5/10
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - loss: 8.6184e-05 - mse: 8.6184e-05 - val_loss: 8.6185e-05 - val_mse: 8.6185e-05
Epoch 6/10
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 9.2013e-05 - mse: 9.2013e-05 - val_loss: 8.6185e-05 - val_mse: 8.6185e-05
Epoch 7/10
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 1.0851e-04 - mse: 1.0851e-04 - val_loss: 8.6185e-05 - val_mse: 8.6185e-05
Epoch 8/10
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step

In [32]:
model.evaluate(X_test, y_test)

338/338 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 8.6188e-05 - mse: 8.6188e-05


[8.61882435856387e-05, 8.61882435856387e-05]